In [87]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import re
from pandas import DataFrame as DF
import time
from pdf2str import extract_text_from_pdf
import os
import numpy as np

from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [88]:
abstracts=pd.read_pickle('abstracts.pickle')
columns={'parent_paper': 'child', 'paper_name': 'parent'}
abstracts = abstracts.rename(columns=columns)

abstracts

,child,parent,paper_url,scholar_url,cite_num,year
0,Inferring Air Quality for Station Location Rec...,A novel long-term air quality forecasting algo...,https://ieeexplore.ieee.org/abstract/document/...,x,0,2017
1,Inferring Air Quality for Station Location Rec...,Matrix Completion with Variational Graph Autoe...,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/citations?user=8DEG...,1,2019
2,Inferring Air Quality for Station Location Rec...,Recovering missing values from corrupted spati...,https://link.springer.com/chapter/10.1007/978-...,https://scholar.google.com/citations?user=VTAS...,5,2017
3,Inferring Air Quality for Station Location Rec...,Sparse mobile crowdsensing: challenges and opp...,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/citations?user=o2I4...,100,2016
4,Inferring Air Quality for Station Location Rec...,Mapping urban environmental noise using smartp...,https://www.mdpi.com/1424-8220/16/10/1692,x,24,2016
5,Inferring Air Quality for Station Location Rec...,Air quality estimation by exploiting terrain f...,https://www.sciencedirect.com/science/article/...,https://scholar.google.com/citations?user=Vxi9...,2,2019
6,Inferring Air Quality for Station Location Rec...,Proposing and investigating PCAMARS as a novel...,https://link.springer.com/article/10.1007/s106...,https://scholar.google.com/citations?user=8_il...,1,2019
7,Inferring Air Quality for Station Location Rec...,Optimizing online task allocation for multi-at...,https://ieeexplore.ieee.org/abstract/document/...,https://scholar.google.com/citations?user=uq55...,18,2018
8,Inferring Air Quality for Station Location Rec...,Spatiotemporal Deep Learning Model for Citywid...,http://s-space.snu.ac.kr/handle/10371/150761,x,0,2019
9,Inferring Air Quality for Station Location Rec...,Estimating potential customers anywhere and an...,https://link.springer.com/chapter/10.1007/978-...,https://scholar.google.com/citations?user=f6SB...,6,2015


In [105]:
# dic={'child': ['a1','a1','b1','b1','b2','b2','c1','c2','c3','c3','c4','c4',],
#      'parent':['b1','b2','c1','c2','c3','c4','d1','d2','d3','d4','d5','d6',]}
# abstracts=DF(dic)
# abstracts

In [98]:
parents=abstracts.parent.unique().tolist()
childs=abstracts.child.unique().tolist()
papers=list(set(childs+parents))
len(parents),len(childs),len(papers)

(12, 7, 13)

In [99]:

paper2outs={paper:[] for paper in papers}

for parent in parents:
    paper2outs[parent]=abstracts[abstracts.parent==parent].child.tolist()

    

In [100]:
paper2ins={paper:[] for paper in papers}

for child in childs:
    paper2ins[child]=abstracts[abstracts.child==child].parent.tolist()

In [101]:
loop_num=10
paper_num=len(papers)
d=0.8

paper2old={}
for paper in papers:
    paper2old[paper]=1/paper_num
    
for i in range(loop_num):
    print(i)
    paper2new={}
    for paper in papers:
        
        in_papers=paper2ins[paper]
        step_by_step=[paper2old[in_paper]/len(paper2outs[in_paper]) for in_paper in in_papers]
        step_by_step=sum(step_by_step)
        
        random_open=(1/paper_num)
        
        paper2new[paper]=d*step_by_step+(1-d)*random_open
    
    paper2old=paper2new
 

0
1
2
3
4
5
6
7
8
9


In [102]:
df_rank=DF({'paper':list(paper2old.keys()),'rank':list(paper2old.values())})
df_rank=df_rank.sort_values(by='rank',ascending=False)
df_rank

,paper,rank
5,a1,0.126646
2,b2,0.079385
4,b1,0.059692
3,c3,0.040000
6,c4,0.040000
0,c2,0.027692
10,c1,0.027692
1,d5,0.015385
7,d2,0.015385
8,d4,0.015385


In [103]:
df_rank['rank'].sum()

0.49341538461538453

In [104]:
# df_rank.loc[42,'paper']